In [140]:
import FABRIC

params = FABRIC.loadArgs('./params/params.yaml')
# __ = FABRIC.genFabDf(params)
# __ = FABRIC.analytic(params)

In [1]:
import pandas as pd
df = pd.read_pickle('/esplabdata/FABRIC/prc/UNIVERSAL_FABRIC.pkl.gz', compression = 'gzip')
dataFrame = df

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import textwrap

def featPlot(params, gpFilter, groupDat):
  
  PRE = groupDat[groupDat["expStage"] == "PRE"]
  POST = groupDat[groupDat["expStage"] == "POST"]
  features = groupDat.iloc[:, df.columns.get_loc("mean_value_ML"):].keys()

  sns.set(style = "whitegrid", context = "talk")
  fig, axes = plt.subplots(nrows = 8, ncols = 9, figsize = (9 * 10, 8 * 10), dpi = 200, facecolor = "white")
  axes = axes.flatten()

  for fIndex, feat in enumerate(features):
    featPre, featPost = np.array(list(PRE[feat])), np.array(list(POST[feat]))
    zsFeatPre, zsFeatPost, threshold = np.abs(stats.zscore(featPre)), np.abs(stats.zscore(featPost)), 1
    idxPre, idxPost = np.where(zsFeatPre > threshold)[0], np.where(zsFeatPost > threshold)[0]

    featPre, featPost = np.delete(featPre, idxPre), np.delete(featPost, idxPost)

    sns.boxplot(data = [featPre, featPost], palette = "pastel", ax = axes[fIndex])
    axes[fIndex].set_xticks([0, 1])
    axes[fIndex].set_xticklabels(['PRE', 'POST'])

    meanPre, stdPre = np.mean(featPre), np.std(featPre)
    meanPost, stdPost = np.mean(featPost), np.std(featPost)

    legendText = [
      f"PRE: (Mean: {meanPre:.3e}, Std: {stdPre:.3e})",
      f"POST: (Mean: {meanPost:.3e}, Std: {stdPost:.3e})",
    ]

    legendColors = ["#ABC9EA", "#EFB792"]
    handles = [plt.Line2D([0], [0], color = color, lw = 4) for color in legendColors]

    axes[fIndex].legend(handles, legendText, loc = 'lower right', fontsize = 12, frameon = True)
    wrappedTitle = "\n".join(textwrap.wrap(f"{feat.replace('_', ' ')}", width = 32))
    axes[fIndex].set_title(wrappedTitle, fontsize = 36, pad = 16)
    axes[fIndex].set_xlabel("PRE vs POST", fontsize = 28, labelpad = 16)
    wrappedYlabel = "\n".join(textwrap.wrap(f"{feat.replace('_', ' ')}", width = 32))
    axes[fIndex].set_ylabel(wrappedYlabel, fontsize = 28, labelpad = 16)

    axes[fIndex].grid(True, axis = "y", linestyle = "--", alpha = 0.7)

  plt.tight_layout(pad = 2.0)
  return plt


def analytic(params):
  
  subParamObject = params["visytic"]
  inpPickleName, funcStat = subParamObject["inpPickleName"], subParamObject["funcStat"]
  inpDirectory, inpFolder = subParamObject["inpDirectory"], subParamObject["inpFolder"]
  outDirectory, outFolder = subParamObject["outDirectory"], subParamObject["outFolder"]
  groupFilter = subParamObject["groupFilter"]
    
  groups = dataFrame.groupby(groupFilter)
  for gpFilter, groupDat in groups:
    plt = featPlot(params, gpFilter, groupDat)
    plt.savefig(os.path.join(outDirectory, outFolder, f"{'_'.join(gpFilter)}.png"))
    plt.close()
    
  return True


analytic(params)

In [16]:
# df = pd.read_pickle('/esplabdata/FABRIC/prc/UNIVERSAL_FABRIC_test.pkl.gz', compression = 'gzip')
dataPacketCol = df["dataPacket"].apply(lambda x: x)
df.head(4)

,subId,expType,expStage,expTrial,expName,stabilityStatus,eyeStatus,dataPacket,mean_value_ML,mean_value_AP,...,critical_time_Diffusion_ML,critical_displacement_Diffusion_ML,short_time_scaling_Diffusion_ML,long_time_scaling_Diffusion_ML,short_time_diffusion_Diffusion_AP,long_time_diffusion_Diffusion_AP,critical_time_Diffusion_AP,critical_displacement_Diffusion_AP,short_time_scaling_Diffusion_AP,long_time_scaling_Diffusion_AP
0,1008,MP,POST,1,Balance,Stable,Closed,subId expType expStage expTrial expName...,5.024536e-16,-3.375670e-16,...,0.804641,0.688811,0.870884,0.115903,0.583397,0.235858,0.314121,0.079499,0.860603,0.469559
1,1008,MP,POST,2,Balance,Stable,Closed,subId expType expStage expTrial expName...,8.836857e-16,5.083445e-15,...,0.458370,0.324977,0.888721,0.265243,0.669159,0.490271,0.698773,0.356775,0.877321,0.443394
2,1008,MP,POST,1,Balance,Stable,Open,subId expType expStage expTrial expName...,4.799809e-16,-4.941233e-17,...,0.482806,0.090048,0.877638,0.110996,0.287261,0.074481,0.158553,0.010712,0.892940,0.526468
3,1008,MP,POST,2,Balance,Stable,Open,subId expType expStage expTrial expName...,-9.242755e-16,4.040379e-17,...,0.222694,0.016171,0.879108,0.443008,0.236145,0.095987,0.566624,0.085990,0.889178,0.096801


In [17]:
pd.concat(df["dataPacket"].head(4).tolist(), ignore_index = True)

,subId,expType,expStage,expTrial,expName,stabilityStatus,eyeStatus,time (s),Fx,Fy,Fz,Mx,My,Mz,COPx,COPy
0,1008,MP,POST,1,Balance,Stable,Closed,0.001,1.442505,-0.152435,756.687511,-13.425294,5.799073,-4.725342,0.066168,-0.109528
1,1008,MP,POST,1,Balance,Stable,Closed,0.002,1.401611,0.162674,755.218761,-13.843751,5.957276,-4.687744,0.088607,-0.168387
2,1008,MP,POST,1,Balance,Stable,Closed,0.003,1.341858,-0.508728,756.500011,-13.707520,5.773194,-4.687988,0.062937,-0.147275
3,1008,MP,POST,1,Balance,Stable,Closed,0.004,1.299622,0.070721,757.375011,-13.692872,5.678223,-4.788330,0.049516,-0.143247
4,1008,MP,POST,1,Balance,Stable,Closed,0.005,1.526489,0.116745,755.437511,-13.713868,5.732666,-4.697022,0.058646,-0.150663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,1008,MP,POST,2,Balance,Stable,Open,29.996,2.952026,-1.920227,749.562511,-11.447266,13.137208,-4.720947,0.859468,0.133716
119996,1008,MP,POST,2,Balance,Stable,Open,29.997,2.279663,-1.827515,750.656261,-11.357911,13.111817,-4.729981,0.853532,0.147845
119997,1008,MP,POST,2,Balance,Stable,Open,29.998,2.333130,-1.828125,750.812511,-11.339356,13.458985,-4.812988,0.899407,0.150631
119998,1008,MP,POST,2,Balance,Stable,Open,29.999,2.236084,-1.737122,750.125011,-11.361817,13.109864,-4.794678,0.854508,0.146252


In [4]:
# import pandas as pd
# dfRaw = pd.read_pickle('/esplabdata/FABRIC/prc/UNIVERSAL_FABRIC_RAW.pkl.gz', compression = 'gzip')
# dfExt = pd.read_pickle('/esplabdata/FABRIC/prc/UNIVERSAL_FABRIC_EXT.pkl.gz', compression = 'gzip')